# Thresholding Updraft Vertical Velocity

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import xoak
import geopandas as gpd
import regionmask

import warnings
warnings.filterwarnings('ignore')

In [2]:
#find directories with the WRF-BCC data
ref_dirts = glob.glob('/home/scratch/WRF_BCC/reflectivity/REFD/mid_century_4p5_FIXED/*')
ref_dirts.sort()
ref_dirts = ref_dirts[:]

uvv_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/W_UP_MAX/mid_century_4p5_FIXED/*')
uvv_dirts.sort()

### Mask US Data

In [3]:
#load the WRF-BCC geog and a random refc dummy file
geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
ds = xr.open_mfdataset('/home/scratch/WRF_BCC/reflectivity/REFD/historical/1990-1991/*.nc')

#merge the files and create needed infomation
ds = xr.merge([ds, geog.squeeze()])
ds = ds.rename({"CLONG": 'lon', 'CLAT': 'lat'})
ds = ds.assign_coords({'x': ds.west_east, 'y': ds.south_north})
ds = ds.assign_coords({'lon': ds.lon, 'lat': ds.lat})

#set the lat-lon as the index
ds.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [4]:
#load an USA shapefile
usa = gpd.read_file("/home/jcorner1/Unidata/shapefiles/smoothing_econus.shp")

#mask the data out
state_mask = regionmask.mask_geopandas(usa, ds.lon, ds.lat)
ma = state_mask.values
ma[~np.isnan(ma)] = 1

### Threshold Data

In [5]:
#iterate through each year (directory)
for dirt_number in range(len(ref_dirts)):
       for month in [10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
        #open all the data within the directory
        print(f'current year: {int(ref_dirts[dirt_number][-4:])-1}, {month}')
        ref_ds = xr.open_mfdataset(glob.glob(f'{ref_dirts[dirt_number]}/*{ref_dirts[dirt_number][-4:]}_*-{str(month).zfill(2)}-*.nc'))
        uvv_ds = xr.open_mfdataset(glob.glob(f'{uvv_dirts[dirt_number]}/*-{str(month).zfill(2)}-*.nc'))

        #grab all values
        refc_val = ref_ds.REFD.values
        uvv_val = uvv_ds.W_UP_MAX.values

        #subset times
        times = ref_ds.Time.values

        #threshold upward vertical velocities, reflectivity, and updraft helicity values. 
        thr_uvv = uvv_ds.where(uvv_val >= 18.0, 0)
        thr_uvv = thr_uvv.where(thr_uvv.W_UP_MAX.values <= 18.0, 1)

        #Give all areas with a UH of 18+ and reflectivity of 50+ a value of 1. 
        thr_val = thr_uvv.W_UP_MAX.values * ma

        #find locations where the value is 1.
        locations = np.where(thr_val >= 1)
        print(f'done thresholding! {len(locations[0])} potential storms')

        #close files
        thr_uvv.close()
        uvv_ds.close()

        #create pandas dataframe
        df = pd.DataFrame(columns=['x', 'y', 'Time', 'UVV'])

        xs = []
        ys = []
        timess=[]
        refcs=[]
        uvvs=[]

        #iterate through all potential center points. 
        for point in range(len(locations[0])):

            #save important attribute values for center points
            time = locations[0][point]
            y = locations[1][point] 
            x = locations[2][point]

            #append values to a list
            str_time = np.datetime_as_string(times[time])
            xs.append(x)
            ys.append(y)
            timess.append(str_time)
            refcs.append(refc_val[time,y,x])
            uvvs.append(uvv_val[time,y,x])

            #df = df.append({'x': x, 'y':y, 'Time':str_time, 'DBZ': refc_val[time,y,x]}, ignore_index=True)

        #add list to pandas dataframe
        df['x'] = xs
        df['y'] = ys
        df['Time'] = timess
        df['DBZ'] = refcs
        df['UVV'] = uvvs

        #Save the dataframe as the csv.
        df.to_csv(f'/home/scratch/jcorner1/syn_sev/dataframes/MC4p5_UVV_{str_time[:4]}_{month}_threshold_dataframe.csv')



current year: 2040, 10
done thresholding! 6989 potential storms
current year: 2040, 11
done thresholding! 2937 potential storms
current year: 2040, 12
done thresholding! 63 potential storms
current year: 2040, 1
done thresholding! 459 potential storms
current year: 2040, 2
done thresholding! 315 potential storms
current year: 2040, 3
done thresholding! 15794 potential storms
current year: 2040, 4
done thresholding! 5744 potential storms
current year: 2040, 5
done thresholding! 53334 potential storms
current year: 2040, 6
done thresholding! 60201 potential storms
current year: 2040, 7
done thresholding! 24019 potential storms
current year: 2040, 8
done thresholding! 16592 potential storms
current year: 2040, 9
done thresholding! 11535 potential storms
current year: 2041, 10
done thresholding! 774 potential storms
current year: 2041, 11
done thresholding! 781 potential storms
current year: 2041, 12
done thresholding! 244 potential storms
current year: 2041, 1
done thresholding! 8450 pote